# 04 MODEL TRAINING

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Get in the correct working directory

In [3]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_ratings_predictions/notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_ratings_predictions'

### Model Experiments

In [6]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

#### Load the data

In [7]:
X_train_path = read_yaml(CONFIG_FILE_PATH).model_training.x_train_path
y_train_path = read_yaml(CONFIG_FILE_PATH).model_training.y_train_path

[ 2024-07-16 19:37:40,186: INFO: common: yaml file: config/config.yaml loaded successfully ]
[ 2024-07-16 19:37:40,192: INFO: common: yaml file: config/config.yaml loaded successfully ]


In [8]:
X_train = pd.read_csv(X_train_path)
y_train = pd.read_csv(y_train_path).values

Complete any remaining data preprocessing steps

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#### Fit several models to the data and evaluate baseline performance

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
def fit_model(model_name, x, y):
    
    model = model_name
    model.fit(x, y)
    
    cv_scores = cross_val_score(model_name, x, y, cv=3, scoring='neg_mean_absolute_error')
    cv_mae_scores = np.abs(cv_scores)

    return cv_mae_scores

#### Model 1 -  K Neighbors Regressor

In [13]:
from sklearn.neighbors import KNeighborsRegressor

In [14]:
knr_mae_scores = fit_model(KNeighborsRegressor(), X_train_scaled, y_train)
print(f"K Neighbors CV MAE scores = {knr_mae_scores}")

K Neighbors CV MAE scores = [0.4037619  0.39497619 0.44602439]


#### Model 2 - Random Forest Regressor

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
rf_mae_scores = fit_model(RandomForestRegressor(), X_train_scaled, y_train.ravel())

In [17]:
print(f"RF CV MAE scores = {rf_mae_scores}")

RF CV MAE scores = [0.44809976 0.43685643 0.41783829]


#### Model 3 - AdaBoost Regressor

In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [19]:
ada_mae_scores = fit_model(AdaBoostRegressor(), X_train_scaled, y_train.ravel())

In [20]:
ada_mae_scores

array([0.43407101, 0.41146628, 0.45739036])

### Create the entity

In [21]:
from dataclasses import dataclass
from pathlib import Path

In [22]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    
    root_dir: Path
    x_train_path: Path
    y_train_path: Path
    model_name: str

### Create configuration manager

In [23]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_training

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            x_train_path=config.x_train_path,
            y_train_path=config.y_train_path,
            model_name=config.model_name)

        return model_trainer_config

### Create the component

In [25]:
from src.StarbucksProject import logger
import joblib

In [26]:
class ModelTrainer:
    
    def __init__(self, config: ModelTrainerConfig):
        
        self.config = config

    def train(self):
                
        X_train = pd.read_csv(self.config.x_train_path)
        y_train = pd.read_csv(self.config.y_train_path)
        
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
            
        
        abr = AdaBoostRegressor()
        abr.fit(X_train_scaled, y_train.values.ravel())
        # knr = KNeighborsRegressor()
        # knr.fit(X_train_scaled, y_train)
        
        joblib.dump(abr, os.path.join(self.config.root_dir, self.config.model_name))

### Create the pipeline

In [27]:
try:

    config = ConfigurationManager()

    model_trainer_config = config.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)

    model_trainer.train()
    
except Exception as e:

    raise e

[ 2024-07-16 19:37:41,602: INFO: common: yaml file: config/config.yaml loaded successfully ]
[ 2024-07-16 19:37:41,605: INFO: common: yaml file: params.yaml loaded successfully ]
[ 2024-07-16 19:37:41,607: INFO: common: yaml file: schema.yaml loaded successfully ]
[ 2024-07-16 19:37:41,609: INFO: common: created directory at: artifacts ]
[ 2024-07-16 19:37:41,609: INFO: common: created directory at: artifacts/model_training ]


### Modularize the code (once working)

- Return to project folder